In [1]:
# 01_grid_search.py
# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

__name__ == '__main__'
if __name__ == '__main__':
	# define dataset
	data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
	print(data)
	# data split
	n_test = 4
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 37.914
 > Model[[(0, 0, 0), (0, 0, 0, 0), 't']] 6.103
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] 0.040
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'c']] 64.038
 > Model[[(0, 0, 1), (0, 0, 0, 0), 't']] 74.745
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'ct']] 1.529
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'n']] 10.000
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'c']] 0.000
 > Model[[(0, 1, 0), (0, 0, 0, 0), 't']] 5.542
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'n']] 9.165
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'c']] 0.000
 > Model[[(0, 1, 1), (0, 0, 0, 0), 't']] 2.427
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'ct']] 0.000
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'ct']] 0.000
 > Model[[(1, 0, 0), (0, 0, 0, 0), 'n']] 6.103
 > Model[[(1, 0, 0), (0, 0, 0, 0), 'c']] 0.000
 > Model[[(1, 0, 0), (0, 0, 0, 0), 't']] 6.103
 > Model[[(1, 0, 0), (0, 0, 0, 0), 

In [2]:
# 02_grid_search_daily_births.py
# grid search sarima hyperparameters for daily female dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('daily-total-female-births.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 165
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 44.930
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 7.718
 > Model[[(0, 0, 0), (0, 0, 0, 0), 't']] 20.364
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] 7.195
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 26.223
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'c']] 7.450
 > Model[[(0, 0, 1), (0, 0, 0, 0), 't']] 14.095
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'n']] 19.048
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'ct']] 7.068
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'c']] 7.364
 > Model[[(0, 0, 2), (0, 0, 0, 0), 't']] 11.748
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'n']] 8.722
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'c']] 8.738
 > Model[[(0, 1, 0), (0, 0, 0, 0), 't']] 8.770
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'ct']] 8.787
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'ct']] 7.050
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'n']] 7.080
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'c']] 7.115
 > Model[[(0, 1, 1), (0, 0, 0, 0), 't']] 7.155
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'n']] 7.014
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'ct']] 7.137
 >

In [3]:
# 03_grid_search_monthly_shampoo_sales.py
# grid search sarima hyperparameters for monthly shampoo sales dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-shampoo-sales.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 491.532
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 235.880
 > Model[[(0, 0, 0), (0, 0, 0, 0), 't']] 81.900
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] 103.880
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 328.076
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'c']] 198.046
 > Model[[(0, 0, 1), (0, 0, 0, 0), 't']] 87.885
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'ct']] 104.414
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'n']] 264.178
 > Model[[(0, 0, 2), (0, 0, 0, 0), 't']] 76.317
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'c']] 158.296
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'n']] 136.761
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'c']] 137.368
 > Model[[(0, 1, 0), (0, 0, 0, 0), 't']] 141.006
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'ct']] 143.827
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'ct']] 92.340
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'n']] 107.044
 > Model[[(0, 1, 1), (0, 0, 0, 0), 't']] 81.041
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'c']] 101.681
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'n']] 67.398
 > Model[[(0, 1, 1), (

In [4]:
# 04_grid_search_monthly_mean_temp.py
# grid search sarima hyperparameters for monthly mean temp dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-mean-temp.csv', header=0, index_col=0)
	data = series.values
	# trim dataset to 5 years
	data = data[-(5*12):]
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs(seasonal=[0, 12])
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 50.075
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'n']] 50.075
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'n']] 32.871
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'n']] 2.195
 > Model[[(0, 0, 0), (0, 0, 2, 12), 'n']] 20.767
 > Model[[(0, 0, 0), (0, 1, 1, 12), 'n']] 2.061
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'n']] 2.166
 > Model[[(0, 0, 0), (0, 1, 2, 12), 'n']] 1.866
 > Model[[(0, 0, 0), (1, 0, 1, 12), 'n']] 2.121
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'n']] 1.647
 > Model[[(0, 0, 0), (1, 1, 1, 12), 'n']] 4.387
 > Model[[(0, 0, 0), (1, 0, 2, 12), 'n']] 2.152
 > Model[[(0, 0, 0), (2, 0, 0, 12), 'n']] 1.731
 > Model[[(0, 0, 0), (1, 1, 2, 12), 'n']] 2.141
 > Model[[(0, 0, 0), (2, 0, 1, 12), 'n']] 2.331
 > Model[[(0, 0, 0), (2, 1, 0, 12), 'n']] 2.138
 > Model[[(0, 0, 0), (2, 0, 2, 12), 'n']] 2.013
 > Model[[(0, 0, 0), (2, 1, 1, 12), 'n']] 2.127
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 8.325
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'c']] 8.325
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'c

In [ ]:
# 05_grid_search_monthly_car_sales.py
# grid search sarima hyperparameters for monthly car sales dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-car-sales.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs(seasonal=[0,6,12])
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 6), 'n']] 18608.652
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 18608.652
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'n']] 18608.652
 > Model[[(0, 0, 0), (0, 0, 1, 6), 'n']] 13642.801
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'n']] 11897.522
 > Model[[(0, 0, 0), (0, 0, 2, 6), 'n']] 8191.126
 > Model[[(0, 0, 0), (0, 1, 0, 6), 'n']] 5798.153
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'n']] 2290.827
 > Model[[(0, 0, 0), (0, 1, 1, 6), 'n']] 3958.642
 > Model[[(0, 0, 0), (0, 1, 1, 12), 'n']] 2286.289
 > Model[[(0, 0, 0), (0, 0, 2, 12), 'n']] 9238.132
 > Model[[(0, 0, 0), (0, 1, 2, 6), 'n']] 3044.366
 > Model[[(0, 0, 0), (1, 0, 0, 6), 'n']] 5951.270
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'n']] 1850.671
 > Model[[(0, 0, 0), (1, 0, 1, 6), 'n']] 3764.214
 > Model[[(0, 0, 0), (1, 0, 1, 12), 'n']] 1771.231
 > Model[[(0, 0, 0), (1, 0, 2, 6), 'n']] 3108.363
 > Model[[(0, 0, 0), (1, 0, 2, 12), 'n']] 2339.839
 > Model[[(0, 0, 0), (1, 1, 0, 6), 'n']] 2242.874
 > Model[[(0, 0, 0), (1, 1, 0, 12), '